## Sensor Tutorial
https://www.youtube.com/watch?v=om8klsBj4rc&t=1165s

In [1]:
import carla 
import math 
import random 
import time  
import numpy as np 
import cv2  

client = carla.Client('172.30.1.101', 2000) 
world = client.get_world() 

bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [2]:
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

In [3]:
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(x=0.6, z=1.6))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) 

camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame)) 

In [4]:
camera.stop() 

In [5]:
import os 
import cv2 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from tools.my_detect import Detect 
from lanedet.utils.config import Config


In [6]:
import torch 
torch.cuda.is_available() 

True

In [7]:
# carla config 
config_file = '/root/works/lanedet/demo/configs/carla_scnn_tusimple.py'

cfg = Config.fromfile(config_file)
cfg.show = False 
cfg.savedir = '/root/works/lanedet'
cfg.load_from = '/root/works/lanedet/demo/checkpoints/scnn_r18_tusimple.pth'
carla_detect = Detect(cfg)

In [8]:
def camera_callback(image, data_dict): 
    # print(type(image))  ->  <class 'carla.libcarla.Image'>
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 


In [9]:
def camera_callback(image, data_dict): 
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 
    img = img[:, :, :3]

    result_img = carla_detect.vis_inference(img, cfg)  
    data_dict['image'] = result_img 
    # data_dict['image'] = np.dstack(result_img, np.zeros((image.height, image.width), 1), dtyp=result_img.dtype) 

In [10]:
image_w = camera_bp.get_attribute("image_size_x").as_int() 
image_h = camera_bp.get_attribute("image_size_y").as_int() 

camera_data = {"image": np.zeros((image_w, image_h, 3))} 

camera.listen(lambda image: camera_callback(image, camera_data)) 

In [11]:
vehicle.set_autopilot(True)

In [12]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE) 
cv2.imshow('RGB Camera', camera_data['image']) 
cv2.waitKey(1) 

while True: 
    cv2.imshow('RGB Camera', camera_data['image']) 

    if cv2.waitKey(1) == ord('q'): 
        break

cv2.destroyAllWindows() 

---

In [9]:
import carla     
import math 
import random 
import time 
import numpy as np 
import cv2 

client.reload_world() 

client = carla.Client('localhost', 2000) 
world = client.get_world() 

bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [10]:
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')

init_spwn_point = random.choice(spawn_points)
print(init_spwn_point)
print(carla.Location(x=1, y=1, z=1)) 

vehicle = world.try_spawn_actor(vehicle_bp, init_spwn_point)    

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

for i in range(100): 
    vehicle_pb = random.choice(bp_lib)




Transform(Location(x=79.055290, y=69.822777, z=0.600000), Rotation(pitch=0.000000, yaw=0.073273, roll=0.000000))
Location(x=1.000000, y=1.000000, z=1.000000)


In [11]:
for bp in bp_lib.filter('sensor'): 
    print(bp.id)

sensor.other.gnss
sensor.camera.dvs
sensor.other.obstacle
sensor.lidar.ray_cast
sensor.camera.semantic_segmentation
sensor.other.radar
sensor.camera.instance_segmentation
sensor.camera.depth
sensor.other.lane_invasion
sensor.camera.rgb
sensor.other.rss
sensor.other.collision
sensor.lidar.ray_cast_semantic
sensor.other.imu
sensor.camera.optical_flow


In [12]:
for bp in bp_lib.filter('camera'): 
    print(bp.id)

sensor.camera.depth
sensor.camera.dvs
sensor.camera.optical_flow
sensor.camera.rgb
sensor.camera.instance_segmentation
sensor.camera.semantic_segmentation


In [13]:
for bp in bp_lib.filter('*vehicle*'): 
    print(bp.id)

vehicle.audi.a2
vehicle.audi.tt
vehicle.ford.ambulance
vehicle.carlamotors.firetruck
vehicle.ford.crown
vehicle.mini.cooper_s
vehicle.lincoln.mkz_2017
vehicle.chevrolet.impala
vehicle.micro.microlino
vehicle.dodge.charger_2020
vehicle.jeep.wrangler_rubicon
vehicle.volkswagen.t2_2021
vehicle.mercedes.sprinter
vehicle.volkswagen.t2
vehicle.tesla.cybertruck
vehicle.kawasaki.ninja
vehicle.tesla.model3
vehicle.diamondback.century
vehicle.vespa.zx125
vehicle.dodge.charger_police_2020
vehicle.mercedes.coupe_2020
vehicle.nissan.patrol_2021
vehicle.audi.etron
vehicle.mini.cooper_s_2021
vehicle.gazelle.omafiets
vehicle.yamaha.yzf
vehicle.bh.crossbike
vehicle.harley-davidson.low_rider
vehicle.toyota.prius
vehicle.seat.leon
vehicle.nissan.patrol
vehicle.nissan.micra
vehicle.mercedes.coupe
vehicle.dodge.charger_police
vehicle.bmw.grandtourer
vehicle.ford.mustang
vehicle.citroen.c3
vehicle.lincoln.mkz_2020
vehicle.carlamotors.carlacola


In [14]:
camera_init_trans = carla.Transform(carla.Location(z=2)) 

camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) 

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle) 

inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation') 
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle) 

depth_camera_bp = bp_lib.find('sensor.camera.depth') 
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle) 

dvs_camera_bp = bp_lib.find('sensor.camera.dvs') 
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=vehicle) 

opt_camera_bp = bp_lib.find('sensor.camera.optical_flow') 
opt_camera = world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to=vehicle) 


In [15]:
def rgb_callback(image, data_dict): 
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

def sem_callback(image, data_dict): 
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

def inst_callback(image, data_dict): 
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

def depth_callback(image, data_dict): 
    image.convert(carla.ColorConverter.LogarithmicDepth) 
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

def opt_callback(data, data_dict): 
    image = data.get_color_coded_flow()
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 
    img[:, :, 3] = 255 
    data_dict['opt_image'] = img 

def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([("x", np.uint16), ("y", np.uint16), ("t", np.int64), ("pol", np.bool)])) 
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8) 
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8) 
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol']*2] = 255 
    data_dict['dvs_image'][:,:,0:3] = dvs_img  

In [17]:
image_w = camera_bp.get_attribute("image_size_x").as_int() 
image_h = camera_bp.get_attribute("image_size_y").as_int() 

sensor_data = {'rgb_image': np.zeros((image_w, image_h, 4)), 
               'sem_image': np.zeros((image_w, image_h, 4)), 
               'inst_image': np.zeros((image_w, image_h, 4)), 
               'depth_image': np.zeros((image_w, image_h, 4)), 
               'opt_image': np.zeros((image_w, image_h, 4)), 
               'dvs_image': np.zeros((image_w, image_h, 4))} 

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE) 

top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1) 
lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1) 
tiled = np.concatenate((top_row, lower_row), axis=0) 

cv2.imshow('All cameras', tiled) 
cv2.waitKey(1) 

camera.listen(lambda image: rgb_callback(image, sensor_data)) 
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
opt_camera.listen(lambda image: opt_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data)) 

while True: 
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1) 
    lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['opt_image'], sensor_data['dvs_image']), axis=1) 
    tiled = np.concatenate((top_row, lower_row), axis=0) 

    cv2.imshow('All cameras', tiled) 

    if cv2.waitKey(1) == ord('q'): 
        break

camera.stop() 
sem_camera.stop()
inst_camera.stop()
depth_camera.stop()
opt_camera.stop()
dvs_camera.stop()
cv2.destroyAllWindows() 

/home/k/self-driving-repo/CARLA_0.9.13_RSS/carla_0913rss_env/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


: 